# jPCA_base.ipynb

This notebook has the code necessary to implement the jPCA algorithm, as described in:
https://static-content.springer.com/esm/art%3A10.1038%2Fnature11129/MediaObjects/41586_2012_BFnature11129_MOESM225_ESM.pdf

We have $ct$ sample points of a time-variant vector signal $X(t) = [x_1(t), \dots, x_n(t)]$. These sample points are in the matrix $X \in \Re^{ct \times n}$.  
We want to find the skew-symmetric matrix $M in \Re^{n \times n}$ that best fits $\dot{X}(t) = X(t) M$.  
In other words, $M$ minimizes $\| \dot{X} - X M \|^2$ with the Frobenius norm.

Since $M$ is skew-symmetric, it will have $n(n-1)/2$ independent entries, which we arrange into the column vector $k$, and solve the equivalent problem using "unrolled" columns. This is expressed as:  
$k^* = \text{armgin}_{k \in \Re^{n(n-1)/2}} \| \dot{x} - \tilde{X} H k \|_2$,  
where $\tilde{X} \in \Re^{(ct \ n) \times n^2}$ is a block matrix with $n$ copies 
of $X$ in its main diagonal, 
$H \in \Re^{n^2 \times n(n-1)/2}$ is a linear transformation that puts the elements of $k$ into the unrolled version of $M$, and $\dot{x}$ is the unrolled version of $\dot{X}$.

The solution to this problem is $k^* = (\tilde{X} H) \backslash \dot{x}$

In [1]:
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp

In [2]:
# Create the data matrix X
# Each row of corresponds to a different time point

# Creating X by evolving a dynamical system
n = 4 # dimensionality of the data
ct = 100 # number of time points
# Using a linear skew-symmetric matrix
mat = np.random.rand(n,n)-0.5
Msk = mat - mat.transpose()

def dt_fun(t, y):
    return np.matmul(Msk, y)

t0 = 0. # initial time of integration
tf = 2. # final time of integration
X0 = 2.*(np.random.rand(n)-0.5) # initial state
t_points = np.linspace(t0, tf, ct)
sol = solve_ivp(dt_fun, [t0,tf], X0, t_eval=t_points)
X = sol.y.transpose()

# standardize the X data
X = X - np.mean(X)
X = X / np.std(X)

# create the block-matrix version of X
Xtilde = sp.linalg.block_diag(*([X]*n))

In [3]:
# Create the H matrix
n = X.shape[1]
ct = X.shape[0]
L = np.zeros((n,n), dtype=int)
c = 0
for row in range(n):
    for col in range(row+1, n):
        L[row, col] = c
        L[col, row] = c
        c += 1
        
H = np.zeros((n*n, int(0.5*n*(n-1))))
for col in range(n):
    for row in range(n):
        if col > row:
            H[n*col+row, L[col,row]] = 1.
        elif row > col:
            H[n*col+row, L[col,row]] = -1.

In [4]:
# Approximate the derivatives of X
Xp = np.zeros_like(X)
t_bit = t_points[1] - t_points[0]
Xp[1:,:] = (X[1:,:] - X[:-1,:]) / t_bit

# prev_row = np.zeros(n)
# for row in range(1, ct):
#     Xp[row-1,:] = (X[row,:] - X[row-1,:]) / t_bit
# Xp[-1,:] = Xp[-2,:]

xp = Xp.flatten('F')

In [5]:
# calculate the solution directly from the formula
kstar = np.matmul(np.linalg.pinv(np.matmul(Xtilde, H)), xp)

# reconstruct the matrix Msk that generated the data
Mstar = np.matmul(H, kstar).reshape(n,n)
print(Mstar)
print(Msk)

[[ 0.         -0.07432403  0.35118555  0.05813048]
 [ 0.07432403  0.         -0.39332199 -0.11159976]
 [-0.35118555  0.39332199  0.          0.06402499]
 [-0.05813048  0.11159976 -0.06402499  0.        ]]
[[ 0.         -0.05025653  0.32709481  0.05849312]
 [ 0.05025653  0.         -0.36885419 -0.13040516]
 [-0.32709481  0.36885419  0.          0.06409437]
 [-0.05849312  0.13040516 -0.06409437  0.        ]]


In [6]:
# Frobenius norm of the matrix difference
# Mstar should be the transposed????
norm1 = np.linalg.norm(Mstar- Msk)
print(norm1)

# Angle between Mstar and Msk
angM= np.arccos( (Mstar*Msk).sum() / (np.linalg.norm(Mstar)*np.linalg.norm(Msk)))
print(angM)

# Frobenius norm of derivatives matrix minus its reconstruction
Xp_rec = np.matmul(X, Mstar)
norm2 = np.linalg.norm(Xp - Xp_rec)
print(norm2)

# Angle between Xp and its reconstruction
ang = np.arccos( (Xp * Xp_rec).sum() / (np.linalg.norm(Xp)*np.linalg.norm(Xp_rec)) )
print(ang)

0.0649932334920391
0.06310185693676823
19.78296180185482
3.0213564336572065


In [7]:
# print H and the auxiliary matrices
print("H =")
print(H, end='\n\n')
print("L =")
print(L, end='\n\n')

k = np.arange(1,int(0.5*n*(n-1)+1))
print("k =")
print(k, end='\n\n')
M = np.matmul(H,k)
print("M =")
print(M, end='\n\n')

H =
[[ 0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]]

L =
[[0 0 1 2]
 [0 0 3 4]
 [1 3 0 5]
 [2 4 5 0]]

k =
[1 2 3 4 5 6]

M =
[ 0. -1. -2. -3.  1.  0. -4. -5.  2.  4.  0. -6.  3.  5.  6.  0.]



In [8]:
# 2) Coefficients of determination
# 2.1) Obtain unconstrained M matrix
M_uncons = np.matmul(np.linalg.pinv(X), Xp)
# 2.2) Reconstruct Xp with M_uncons
Xp_uncons = np.matmul(X, M_uncons)
# 2.3) Reconstruct Xp with Mstar
Xp_skew = np.matmul(X, Mstar)
# 2.4) Calculate residual sums of squares
SSres_uncons = ((Xp - Xp_uncons) * (Xp - Xp_uncons)).sum()
SSres_skew = ((Xp - Xp_skew) * (Xp - Xp_skew)).sum()
# 2.5) Calculate the total sum of squares
SStot = ((Xp-Xp.mean())*(Xp-Xp.mean())).sum()
# 2.6) Calculate coefficients of determination
R2_uncons = 1. - (SSres_uncons / SStot)
R2_skew = 1. - (SSres_skew / SStot)

print("R2 unconstrained: %f" % (R2_uncons))
print("R2 skew symmetric: %f" % (R2_skew))

R2 unconstrained: 0.990658
R2 skew symmetric: -3.292867


In [9]:
a = np.array([[0,1],[-1,0]])
eig_vals, eig_vecs = np.linalg.eig(a)
print(a)
print("Eigenvalues: ")
print(eig_vals)
print("Eigenvectors: ")
print(eig_vecs)

[[ 0  1]
 [-1  0]]
Eigenvalues: 
[0.+1.j 0.-1.j]
Eigenvectors: 
[[0.70710678+0.j         0.70710678-0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]
